In [1]:
# | echo: false
import os
import subprocess
from os import fspath
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from osgeo import gdal


%load_ext autoreload
%autoreload 2

%matplotlib inline

# Dolphin basic walkthrough

This notebook demonstrates the basic usage of the `dolphin` command line tool to execute the stack-based phase linking workflow.
In this notebook, we will

- Prepare a configuration file for a stack of coregistered single-look complex (SLC) radar images with `dolphin config`
- Run this configuration file with `dolphin run` 
- Inspect the resulting output interferograms
- Show the most common parameters you may want to change or customize



## Setup

We first need to install `dolphin` as outlined in the [Getting Started](https://dolphin-insar.readthedocs.io/en/getting-started) section of the documentation. 
We can check that we have the command line tool correctly installed by running

In [2]:
# | echo: true
!dolphin --help

usage: dolphin [-h] {run,config} ...

options:
  -h, --help    show this help message and exit

subcommands:
  {run,config}


or by importing dolphin in python

In [3]:
# | echo: true
import dolphin

print(dolphin.__version__)

0.0.3.post1.dev22+gd29c021.d20230202


If you have a GPU available to you, you can follow the extra installation set up so that the GPU verion of the workflow run.
This can be 5-20x faster than the CPU version, depending on the sie of your workstation.

## Input dataset

We will use a stack of Sentinel-1 SLCs from descending track 87. 
These were produced by [COMPASS](https://github.com/opera-adt/COMPASS) and are available for download on Zenodo (TODO).

A brief walkthrough of how to produce these is included in the Appendix

In [4]:
!mkdir -p data/hawaii
os.chdir("data/hawaii")

dolphin_config.yaml output              scratch             slcs


In the `slc` directory, we have stored the NetCDF-format SLCs:

In [7]:
!ls slcs

t087_185683_iw2_20180210_VV.h5 t087_185684_iw2_20180210_VV.h5
t087_185683_iw2_20180222_VV.h5 t087_185684_iw2_20180222_VV.h5
t087_185683_iw2_20180306_VV.h5 t087_185684_iw2_20180306_VV.h5
t087_185683_iw2_20180318_VV.h5 t087_185684_iw2_20180318_VV.h5
t087_185683_iw2_20180330_VV.h5 t087_185684_iw2_20180330_VV.h5
t087_185683_iw2_20180411_VV.h5 t087_185684_iw2_20180411_VV.h5
t087_185683_iw2_20180423_VV.h5 t087_185684_iw2_20180423_VV.h5
t087_185683_iw2_20180505_VV.h5 t087_185684_iw2_20180505_VV.h5
t087_185683_iw2_20180511_VV.h5 t087_185684_iw2_20180511_VV.h5
t087_185683_iw2_20180517_VV.h5 t087_185684_iw2_20180517_VV.h5
t087_185683_iw2_20180523_VV.h5 t087_185684_iw2_20180523_VV.h5
t087_185683_iw2_20180529_VV.h5 t087_185684_iw2_20180529_VV.h5
t087_185683_iw2_20180604_VV.h5 t087_185684_iw2_20180604_VV.h5
t087_185683_iw2_20180610_VV.h5 t087_185684_iw2_20180610_VV.h5
t087_185683_iw2_20180616_VV.h5 t087_185684_iw2_20180616_VV.h5


We have 30 total files from 2018. The naming convention comes from COMPASS, where, for example, `t087_185683_iw2_20180210_VV.h5` means
- `t087` is Sentinel-1 track 87
- 185683 the Burst IDs from [ESA's Burst database](https://sentinel.esa.int/web/sentinel/-/publication-of-brust-id-maps-for-copernicus-sentinel-1/1.1).
- `iw2` indicates these are from the middle subswath (since the "Burst ID" is repeated for subswaths IW1,2,3.)
- `20180210` is the acquisition date
- `VV` is the polarization

We have 15 dates from the two adjacent bursts `185683` and `185684`. 
We will first process just one stack for demonstration; however, you can input multiple burst stacks from COMPASS into `dolphin`, and it will form burst-wise interferograms and stitch them before unwrapping.

Let's make a configuration file for all of the 185683 bursts:

In [10]:
!dolphin config --slc-files slcs/*185683*.h5

[02/03 11:00:10] [INFO config.py] CSLC files look like OPERA files, setting subdataset to science/SENTINEL1/CSLC/grids/VV.
Saving configuration to dolphin_config.yaml


If you need more fine-grained control of which SLCs to include, you can list the file locations in a text file separated by new lines and refer to it with an `@` symbol. For example: 

In [19]:
# Store the files we want in a text file called slc_list.txt
# Here we're just using `ls` to all the 185683 SLCs
!ls slcs/*185683* > slc_list.txt

# We use the same `--slc-files` argument, but now use an @ to say look inside the file
!dolphin config --slc-files @slc_list.txt -o new_config.yaml

[02/03 11:10:33] [INFO config.py] CSLC files look like OPERA files, setting subdataset to science/SENTINEL1/CSLC/grids/VV.
Saving configuration to new_config.yaml


This is an equivalent way to point to the SLCs you want to process. The configs should be the same (except for the creation time, which is logged):

In [20]:
!diff new_config.yaml dolphin_config.yaml

74c74
< creation_time_utc: '2023-02-03T16:10:33.742851'
---
> creation_time_utc: '2023-02-03T16:00:10.885488'


This command created a YAML file in our current directory. Most of the contents were filled in by the workflow defaults:

In [12]:
!cat dolphin_config.yaml

workflow_name: stack
inputs:
  cslc_file_list:
  - slcs/t087_185683_iw2_20180210_VV.h5
  - slcs/t087_185683_iw2_20180222_VV.h5
  - slcs/t087_185683_iw2_20180306_VV.h5
  - slcs/t087_185683_iw2_20180318_VV.h5
  - slcs/t087_185683_iw2_20180330_VV.h5
  - slcs/t087_185683_iw2_20180411_VV.h5
  - slcs/t087_185683_iw2_20180423_VV.h5
  - slcs/t087_185683_iw2_20180505_VV.h5
  - slcs/t087_185683_iw2_20180511_VV.h5
  - slcs/t087_185683_iw2_20180517_VV.h5
  - slcs/t087_185683_iw2_20180523_VV.h5
  - slcs/t087_185683_iw2_20180529_VV.h5
  - slcs/t087_185683_iw2_20180604_VV.h5
  - slcs/t087_185683_iw2_20180610_VV.h5
  - slcs/t087_185683_iw2_20180616_VV.h5
  subdataset: science/SENTINEL1/CSLC/grids/VV
  cslc_date_fmt: '%Y%m%d'
  mask_files: []
outputs:
  output_format: NetCDF
  scratch_directory: /Users/staniewi/repos/dolphin/notebooks/data/hawaii/scratch
  output_directory: /Users/staniewi/repos/dolphin/notebooks/data/hawaii/output
  output_resolution:
  strides:
    x: 1
    y: 1
  hdf5_creation_optio

## Running the workflow

Now that we have created the `dolphin_config.yaml` file, we can run it using `dolphin run`

In [15]:
!dolphin run dolphin_config.yaml

[02/03 11:07:36] [INFO s1_disp_stack.py] Running wrapped phase estimation
[02/03 11:07:36] [INFO wrapped_phase.py] Creating persistent scatterer file /Users/staniewi/repos/dolphin/notebooks/data/hawaii/scratch/PS/ps_pixels.tif
100%|███████████████████████████████████████████| 10/10 [00:41<00:00,  4.14s/it]
[02/03 11:08:19] [INFO ps.py] Waiting to write 0 blocks of data.
[02/03 11:08:19] [INFO ps.py] Finished writing out PS files
[02/03 11:08:19] [INFO wrapped_phase.py] Running sequential EMI step in /Users/staniewi/repos/dolphin/notebooks/data/hawaii/scratch/linked_phase
[02/03 11:08:19] [INFO sequential.py] VRTStack(15 bands, outfile=/Users/staniewi/repos/dolphin/notebooks/data/hawaii/scratch/slc_stack.vrt): from /Users/staniewi/repos/dolphin/notebooks/data/hawaii/slcs/t087_185683_iw2_20180210_VV.h5 to /Users/staniewi/repos/dolphin/notebooks/data/hawaii/slcs/t087_185683_iw2_20180616_VV.h5
[02/03 11:08:20] [INFO sequential.py] Processing 15 files + 0 compressed. Output folder: /Users/s

## Appendix: Producing stacks of Sentinel-1 geocoded SLCs

TODO: show the commands for `s1_geocode_stack`

- DEM download
- ASF download
- `eof`
- burst DB, and select bursts

```
~/repos/COMPASS/src/compass/s1_geocode_stack.py -d ../dem.tif --orbit-dir ../orbits/ -s ../data/ -w stack-no-lut  -b `paste -sd' ' < bursts.txt` -ed 20180901
```